# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io

from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy.io
from skfeature.function.sparse_learning_based import MCFS
from skfeature.utility import construct_W

import time
import pandas as pd

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
#--------------------------------------------------------------------------------------------------------------------------------
def ETree(p_train_feature,p_train_label,p_test_feature,p_test_label,p_seed):
    clf = ExtraTreesClassifier(n_estimators=50, random_state=p_seed)
    
    # Training
    clf.fit(p_train_feature, p_train_label)
    
    # Training accuracy
    print('Training accuracy：',clf.score(p_train_feature, np.array(p_train_label)))
    print('Training accuracy：',accuracy_score(np.array(p_train_label),clf.predict(p_train_feature)))
    #print('Training accuracy：',np.sum(clf.predict(p_train_feature)==np.array(p_train_label))/p_train_label.shape[0])

    # Testing accuracy
    print('Testing accuracy：',clf.score(p_test_feature, np.array(p_test_label)))
    print('Testing accuracy：',accuracy_score(np.array(p_test_label),clf.predict(p_test_feature)))
    #print('Testing accuracy：',np.sum(clf.predict(p_test_feature)==np.array(p_test_label))/p_test_label.shape[0])

In [3]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')

# 2. Loading data

In [4]:
data_path="./Dataset/arcene.mat"
Data = scipy.io.loadmat(data_path)

data_arr=Data['X']
label_arr_=Data['Y'][:, 0]

label_arr=np.array([0 if label_arr_i<0 else label_arr_i for label_arr_i in label_arr_])

data_arr=MinMaxScaler(feature_range=(0,1)).fit_transform(data_arr)

C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(data_arr,label_arr,test_size=0.2,random_state=seed)

print('Shape of C_train_x: ' + str(C_train_x.shape)) 
print('Shape of C_train_y: ' + str(C_train_y.shape)) 
print('Shape of C_test_x: ' + str(C_test_x.shape)) 
print('Shape of C_test_y: ' + str(C_test_y.shape)) 

Shape of C_train_x: (160, 10000)
Shape of C_train_y: (160,)
Shape of C_test_x: (40, 10000)
Shape of C_test_y: (40,)


In [5]:
key_feture_number=64

# 3. Classifying 1

### Extra Trees

In [6]:
train_feature=C_train_x
train_label=C_train_y
test_feature=C_test_x
test_label=C_test_y

print('Shape of train_feature: ' + str(train_feature.shape)) 
print('Shape of train_label: ' + str(train_label.shape)) 
print('Shape of test_feature: ' + str(test_feature.shape)) 
print('Shape of test_label: ' + str(test_label.shape)) 

p_seed=seed
ETree(train_feature,train_label,test_feature,test_label,p_seed)

Shape of train_feature: (160, 10000)
Shape of train_label: (160,)
Shape of test_feature: (40, 10000)
Shape of test_label: (40,)
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.85
Testing accuracy： 0.85


In [7]:
num_cluster=len(np.unique(label_arr))

# 4. Model

In [8]:
start = time.clock()

# construct affinity matrix
kwargs_W =  {"metric": "euclidean", "neighborMode": "knn", "weightMode": "heatKernel", "k": 5, 't': 1}

train_W = construct_W.construct_W(train_feature, **kwargs_W)

# obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
tain_score = MCFS.mcfs(train_feature, n_selected_features=key_feture_number, W=train_W,n_clusters=num_cluster)
train_idx = MCFS.feature_ranking(tain_score)

# obtain the dataset on the selected features
train_selected_x = train_feature[:, train_idx[0:key_feture_number]]

print("train_selected_x",train_selected_x.shape)


test_W = construct_W.construct_W(test_feature, **kwargs_W)

# obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
test_score = MCFS.mcfs(test_feature, n_selected_features=key_feture_number, W=test_W,n_clusters=num_cluster)
test_idx = MCFS.feature_ranking(test_score)

# obtain the dataset on the selected features
test_selected_x = test_feature[:, test_idx[0:key_feture_number]]

print("Data_selected_x",test_selected_x.shape)


time_cost=time.clock() - start

write_to_csv(np.array([time_cost]),"./log/MCFS_time"+str(key_feture_number)+".csv")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


train_selected_x (160, 64)
Data_selected_x (40, 64)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


# 5. Classifying 2

### Extra Trees

In [9]:
train_feature=train_selected_x
train_label=C_train_y

test_feature=test_selected_x
test_label=C_test_y

print('Shape of train_feature: ' + str(train_feature.shape)) 
print('Shape of train_label: ' + str(train_label.shape)) 
print('Shape of test_feature: ' + str(test_feature.shape)) 
print('Shape of test_label: ' + str(test_label.shape)) 

p_seed=seed
ETree(train_feature,train_label,test_feature,test_label,p_seed)

Shape of train_feature: (160, 64)
Shape of train_label: (160,)
Shape of test_feature: (40, 64)
Shape of test_label: (40,)
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.625
Testing accuracy： 0.625


# 6. Reconstruction loss

In [10]:
from sklearn.linear_model import LinearRegression

def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR

In [11]:
train_feature_tuple=(train_selected_x,C_train_x)
test_feature_tuple=(test_selected_x,C_test_x)

reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
print(reconstruction_loss)

4.826201016478801
